## Beat Tracking Application

In [3]:
import numpy as np
from beattracker.utilities import load_file
from beattracker.onsetdetector import OnsetDetector
from beattracker.clustering import cluster_ioi
from beattracker.agentsystem import agent_track
from beattracker.downbeats import get_downbeats

In [4]:
def BeatTracker(filepath):
    """Returns predictions and downbears as numpy arrays of event times
    Args:
        filepath str: path to file
    Returns:
        np.array, np.array: prediction beats and downbeats
    """
    #Parameters used in different components of the systems
    peak_picking_params = {'wd': 1,
                    'lam': 0.5499999999999999,
                    'al': 0.9599999999999999,
                    'w': 39,
                    'm': 1,
                    'bpm': False,
                    'win_sec': 0.04,
                    'hop_sec': 0.01,
                    'odfs': ['SF']}
    
    cluster_ioi_params = {'cw': .025, 'bpm_l': 40, 'bpm_u': 230}

    agent_params = {'su':5, 'iw': .040, 'cf':39, 'to': 5}
    
    #Load File
    x, sr = load_file(filepath)
    
    #Create OnsetDectector object
    onset_detector = OnsetDetector(x, sr, peak_picking_params)
    
    #Sets onsets
    onset_detector.set_onsets()
    
    #1 dim np array of pairwise distances between onsets
    iois = onset_detector.get_iois()
    
    #1 dim np array of onset event times
    events = onset_detector.get_peak_times()
    
    #1 dim np array of indices that correspond onset poisitions in the full odf
    event_odf_ind = onset_detector.get_peak_indexs()
    
    #1 dim np array of spectral flux values corresponding to events. Used for salience scoring
    odf_sal = onset_detector.get_peak_odf()
    
    #scalar value representing the odf frame length in seconds
    frame_sec = onset_detector.get_frame_secs()
    
    #Returns the 5 cluster centers of iois with the highest scores
    clusters = cluster_ioi(iois, cluster_ioi_params)
    
    #runs agent system. returns a list of beat times in seconds as well as the indices of the events in the full odf
    preds, pred_ind  = agent_track(clusters, events, odf_sal, frame_sec, event_odf_ind, agent_params)
    
    if len(preds) == 0:
        downbeats = np.array([])
    else:
        # get the full normalized odf used to score salinece
        odf_sal_norm_full = onset_detector.get_odf_norm()
        downbeats = get_downbeats(preds, pred_ind, odf_sal_norm_full)
    
    return preds, downbeats
    

In [5]:
preds, downbeats = BeatTracker('path/to/file.wav')

/Users/nickreeves/Desktop/MSC/Sem_2/MI/CW1_BT/Submission/beattracker/odf.py:57: ComplexWarning: Casting complex values to real discards the imaginary part
  odf[sf_index,frame_num] = sf
